In [1]:
!pip install wfdb
!pip install hrv-analysis
!pip install py-ecg-detectors
!pip install neurokit2

from google.colab import output
output.clear()
print("Packages installed successfully")

Packages installed successfully


In [2]:
import pprint
import os
import datetime

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import wfdb
import hrvanalysis
import ecgdetectors
import hrv
import neurokit2 as nk

In [3]:
# DOWNLOAD_PATH = "https://physionet.org/content/nsr2db/1.0.0/"
# SAVE_DIR = "/content/drive/MyDrive/Projects/HRV/dataset"
# !wget -r -N -c -np $DOWNLOAD_PATH -P $SAVE_DIR

# !wget -r -N -c -np https://physionet.org/files/nsr2db/1.0.0/ -P /content/dataset

In [4]:
# NSR2DB_COLAB_DIR = ...
# NSR2DB_GDRIVE_DIR = "/content/drive/MyDrive/Projects/HRV/dataset/physionet.org/files/"
# !cp -R /content/dataset/physionet.org/files/* $NSR2DB_GDRIVE_DIR

## Normal Sinus Rhythm RR Interval Database

- https://physionet.org/content/nsr2db/1.0.0/

In [5]:
rec = wfdb.get_record_list('nsr2db')
print(rec)

['nsr001', 'nsr002', 'nsr003', 'nsr004', 'nsr005', 'nsr006', 'nsr007', 'nsr008', 'nsr009', 'nsr010', 'nsr011', 'nsr012', 'nsr013', 'nsr014', 'nsr015', 'nsr016', 'nsr017', 'nsr018', 'nsr019', 'nsr020', 'nsr021', 'nsr022', 'nsr023', 'nsr024', 'nsr025', 'nsr026', 'nsr027', 'nsr028', 'nsr029', 'nsr030', 'nsr031', 'nsr032', 'nsr033', 'nsr034', 'nsr035', 'nsr036', 'nsr037', 'nsr038', 'nsr039', 'nsr040', 'nsr041', 'nsr042', 'nsr043', 'nsr044', 'nsr045', 'nsr046', 'nsr047', 'nsr048', 'nsr049', 'nsr050', 'nsr051', 'nsr052', 'nsr053', 'nsr054']


In [6]:
# NSR2DB_DIR = "/content/dataset/physionet.org/files/nsr2db/1.0.0/"
NSR2DB_DIR = "/content/drive/MyDrive/Projects/HRV/dataset/physionet.org/files/nsr2db/1.0.0/"

In [7]:
FS = 128
h = wfdb.rdheader(os.path.join(NSR2DB_DIR, "nsr002"))
h.__dict__

{'record_name': 'nsr002',
 'n_sig': 0,
 'fs': 128,
 'counter_freq': None,
 'base_counter': None,
 'sig_len': 0,
 'base_time': None,
 'base_date': None,
 'comments': ['Age: 67 Sex: M'],
 'sig_name': None,
 'p_signal': None,
 'd_signal': None,
 'e_p_signal': None,
 'e_d_signal': None,
 'file_name': None,
 'fmt': None,
 'samps_per_frame': None,
 'skew': None,
 'byte_offset': None,
 'adc_gain': None,
 'baseline': None,
 'units': None,
 'adc_res': None,
 'adc_zero': None,
 'init_value': None,
 'checksum': None,
 'block_size': None}

In [8]:
ann = wfdb.rdann(os.path.join(NSR2DB_DIR, "nsr002"), extension='ecg')
pprint.pprint(ann.__dict__.keys())

dict_keys(['record_name', 'extension', 'sample', 'symbol', 'subtype', 'chan', 'num', 'aux_note', 'fs', 'label_store', 'description', 'custom_labels', 'contained_labels', 'ann_len'])


In [9]:
# ann.__dict__

In [16]:
from google.colab import output

start_time = datetime.datetime.now()

FS = 128

nsr2db = {}
for record in wfdb.get_record_list('nsr2db'):
    try:
        print("Record:", record)
        ann = wfdb.rdann(os.path.join(NSR2DB_DIR, record), extension='ecg')
        record_hea = wfdb.rdheader(os.path.join(NSR2DB_DIR, record))

        r_peaks = np.array(ann.sample, dtype=np.float32)
        annotation = np.array(ann.symbol)

        # counting ectopic beats
        abnormal_beat_num = 0
        for _sym, _count in np.transpose(np.unique(annotation, return_counts=True)):
            if _sym != 'N':
                abnormal_beat_num += int(_count)
            if _sym == 'N':
                normal_beat_num = int(_count)
        normal_beats_ratio = normal_beat_num / (normal_beat_num + abnormal_beat_num)

        # check & replace ectopic beat with np.nan
        for idx, ann in enumerate(annotation):
            if str(ann).upper() != 'N':
                r_peaks[idx] = np.nan
        # interpolate
        r_peaks = hrvanalysis.preprocessing.interpolate_nan_values(r_peaks)

        # retrieve rr_interval from distance between 2 r_peak points
        rri = [r_peaks[i+1] - r_peaks[i] for i in range(len(r_peaks) - 1)]
        # convert the unit from freq_sample to milisecond
        rri = list(np.array(rri) * 1000 / FS)
        print("  len(r_peaks)           :", len(r_peaks))
        print("  len(rri)               :", len(rri))

        rri = hrvanalysis.preprocessing.remove_outliers(
            rri,
            low_rri = 300,
            high_rri = 2000,
        )
        nni = hrvanalysis.preprocessing.interpolate_nan_values(rri)

        # remove nan values if exists
        nni = [val for val in nni if np.isfinite(val)]
        print("  len(nni)               :", len(nni))
        time_domain = hrvanalysis.extract_features.get_time_domain_features(nni)
        print("  time_domain computed!")
        freq_domain = hrvanalysis.extract_features.get_frequency_domain_features(nni)
        print("  freq_domain computed!")
        geom = hrvanalysis.extract_features.get_geometrical_features(nni)
        print("  geometrical features computed!")
        poincare = hrvanalysis.extract_features.get_poincare_plot_features(nni)
        print("  poincare features computed!")
        csi_cvi = hrvanalysis.extract_features.get_csi_cvi_features(nni)
        print("  csi cvi computed!")
        # Function computing the sample entropy of the given data
        # Must use this function on short term recordings, from 1 minute window
        if len(nni) <= 200:
            # Ref: https://ieeexplore.ieee.org/document/8295257
            # Sample entropy (SampEn), a popularly used “regularity analysis” tool,
            # has restrictions in handling shortterm segments (largely N ≤ 200) of
            # heart rate variability (HRV) data
            sampen = hrvanalysis.extract_features.get_sampen(nni)
            print("  sampen computed!")
        else:
            print("  nn > 200")
            print("  nn is set to NaN")
            sampen = {"sampen": np.nan}

        nsr2db[record] = {
            "id": "nsr2db-" + str(record),
            "db_source": "nsr2db",
            "age": record_hea.comments[0].split()[1],
            "gender": record_hea.comments[0].split()[3],
            "fs": FS,
            "signal_length": np.nan,
            "recording_time_hours": np.nan,
            "recording_time_seconds": np.nan,
            "normal_beats": normal_beat_num,
            "abnormal_beats": abnormal_beat_num,
            "normal_beats_ratio": normal_beats_ratio,
            "rri_length": len(rri),
            "nni_length": len(nni),
            **time_domain,
            **freq_domain,
            **geom,
            **poincare,
            **csi_cvi,
            **sampen,
            "cardiac_info": "Sinus",
            "risk": 0
        }
        print("  RECORD {} SAVED!".format(record))

        output.clear()
    except:
        print("\nERROR (%s)\n" %record)
        continue

print(datetime.datetime.now() - start_time)

0:03:42.281004


In [17]:
# pprint.pprint(nsr2db)

In [18]:
df_nsr2db = pd.DataFrame(nsr2db).T
df_nsr2db

,id,db_source,age,gender,fs,signal_length,recording_time_hours,recording_time_seconds,normal_beats,abnormal_beats,normal_beats_ratio,rri_length,nni_length,mean_nni,sdnn,sdsd,nni_50,pnni_50,nni_20,pnni_20,rmssd,median_nni,range_nni,cvsd,cvnni,mean_hr,max_hr,min_hr,std_hr,lf,hf,lf_hf_ratio,lfnu,hfnu,total_power,vlf,triangular_index,tinn,sd1,sd2,ratio_sd2_sd1,csi,cvi,Modified_csi,sampen,cardiac_info,risk
nsr001,nsr2db-nsr001,nsr2db,64,F,128,NaN,NaN,NaN,106379,456,0.995732,106834,106834,758.344429,170.819356,39.25047,9754,9.130138,37691,35.280297,39.25047,703.125,1167.96875,0.051758,0.225253,83.057407,199.480519,40.851064,18.258777,778.630879,428.132798,1.818667,64.522234,35.477766,2330.796029,1124.032351,35.096583,None,27.754403,239.975411,8.646391,8.646391,5.027619,8299.68461,NaN,Sinus,0
nsr002,nsr2db-nsr002,nsr2db,67,M,128,NaN,NaN,NaN,111100,284,0.99745,111383,111382,754.286171,129.156802,19.693105,1410,1.265925,21823,19.593108,19.693105,742.1875,1445.3125,0.026108,0.171231,81.876288,199.480519,34.362416,13.870781,523.537577,49.767639,10.519639,91.319172,8.680828,1310.42723,737.122013,29.411672,None,13.925191,182.123716,13.078723,13.078723,4.608288,9527.78283,NaN,Sinus,0
nsr003,nsr2db-nsr003,nsr2db,67,F,128,NaN,NaN,NaN,97274,581,0.994063,97854,97854,865.172381,107.611129,46.198889,9144,9.344629,42736,43.673674,46.198889,890.625,1664.0625,0.053398,0.124381,70.6112,196.923077,30.47619,10.654971,736.367703,421.928603,1.745242,63.573345,36.426655,1990.267463,831.971157,21.08468,None,32.667714,148.637581,4.549984,4.549984,4.890367,2705.194542,NaN,Sinus,0
nsr004,nsr2db-nsr004,nsr2db,62,F,128,NaN,NaN,NaN,97779,64,0.999346,97842,97842,850.729445,172.722482,30.805785,8513,8.700851,41323,42.23485,30.805786,843.75,1207.03125,0.036211,0.203029,73.669548,196.923077,39.689922,15.929262,714.41616,262.515948,2.72142,73.128537,26.871463,1933.27466,956.342552,34.008342,None,21.783091,243.293256,11.168904,11.168904,4.92837,10869.276197,NaN,Sinus,0
nsr005,nsr2db-nsr005,nsr2db,62,F,128,NaN,NaN,NaN,116261,1033,0.991193,117293,117293,733.69418,221.550335,32.689939,6200,5.285953,39612,33.772124,32.689939,679.6875,1496.09375,0.044555,0.301966,89.109575,199.480519,33.391304,25.261726,1259.575385,203.648133,6.185057,86.082227,13.917773,2771.409102,1308.185583,21.83414,None,23.115376,312.465648,13.517654,13.517654,5.062823,16895.209705,NaN,Sinus,0
nsr006,nsr2db-nsr006,nsr2db,64,M,128,NaN,NaN,NaN,102620,431,0.995818,103050,103050,806.669927,167.963744,31.154222,2646,2.567711,26687,25.897389,31.154222,835.9375,1644.53125,0.038621,0.208219,78.277332,199.480519,30.843373,19.305681,828.984763,190.137039,4.359933,81.343051,18.656949,1761.326632,742.204831,32.203125,None,22.029468,236.512878,10.736205,10.736205,4.920979,10157.002522,NaN,Sinus,0
nsr007,nsr2db-nsr007,nsr2db,76,M,128,NaN,NaN,NaN,106702,779,0.992752,107480,107480,786.528235,107.585783,41.329094,1690,1.5724,27806,25.8711,41.329094,789.0625,1699.21875,0.052546,0.136786,78.007762,199.480519,30.0,13.396506,681.793431,315.684604,2.15973,68.351724,31.648276,1545.334328,547.856293,19.779168,None,29.224219,149.316264,5.109333,5.109333,4.84397,3051.626035,NaN,Sinus,0
nsr008,nsr2db-nsr008,nsr2db,64,F,128,NaN,NaN,NaN,107665,381,0.996474,108045,108045,794.690472,147.34658,34.902264,5904,5.46444,32964,30.509792,34.902264,765.625,1371.09375,0.043919,0.185414,78.137279,199.480519,35.88785,14.812643,987.308842,258.411607,3.820683,79.256052,20.743948,2244.151944,998.431495,32.445946,None,24.679742,206.912879,8.383916,8.383916,4.912248,6938.960738,NaN,Sinus,0
nsr009,nsr2db-nsr009,nsr2db,66,M,128,NaN,NaN,NaN,102829,45,0.999563,102873,102873,836.051643,167.950087,41.501479,11207,10.894121,38002,36.941053,41.501479,804.6875,1246.09375,0.04964,0.200885,74.671396,199.480519,38.787879,14.903639,1924.390666,380.755886,5.054132,83.482357,16.517643,3671.471186,1366.324634,30.87425,None,29.34612,235.697409,8.031638,8.031638,5.044025,7572.145042,NaN,Sinus,0
nsr010,nsr2db-nsr010,nsr2db,61,F,128,NaN,NaN,NaN,96045,246,0.997445,962

In [19]:
df_nsr2db.to_csv("df_nsr2db.csv", index=False)
df_nsr2db.to_excel("df_nsr2db.xlsx", index=False)

In [20]:
pd.set_option("display.max_columns", None)

In [22]:
pd.read_csv("df_nsr2db.csv")

,id,db_source,age,gender,fs,signal_length,recording_time_hours,recording_time_seconds,normal_beats,abnormal_beats,normal_beats_ratio,rri_length,nni_length,mean_nni,sdnn,sdsd,nni_50,pnni_50,nni_20,pnni_20,rmssd,median_nni,range_nni,cvsd,cvnni,mean_hr,max_hr,min_hr,std_hr,lf,hf,lf_hf_ratio,lfnu,hfnu,total_power,vlf,triangular_index,tinn,sd1,sd2,ratio_sd2_sd1,csi,cvi,Modified_csi,sampen,cardiac_info,risk
0,nsr2db-nsr001,nsr2db,64.0,F,128,NaN,NaN,NaN,106379,456,0.995732,106834,106834,758.344429,170.819356,39.250470,9754,9.130138,37691,35.280297,39.250470,703.1250,1167.968750,0.051758,0.225253,83.057407,199.480519,40.851064,18.258777,778.630879,428.132798,1.818667,64.522234,35.477766,2330.796029,1124.032351,35.096583,NaN,27.754403,239.975411,8.646391,8.646391,5.027619,8299.684610,NaN,Sinus,0
1,nsr2db-nsr002,nsr2db,67.0,M,128,NaN,NaN,NaN,111100,284,0.997450,111383,111382,754.286171,129.156802,19.693105,1410,1.265925,21823,19.593108,19.693105,742.1875,1445.312500,0.026108,0.171231,81.876288,199.480519,34.362416,13.870781,523.537577,49.767639,10.519639,91.319172,8.680828,1310.427230,737.122013,29.411672,NaN,13.925191,182.123716,13.078723,13.078723,4.608288,9527.782830,NaN,Sinus,0
2,nsr2db-nsr003,nsr2db,67.0,F,128,NaN,NaN,NaN,97274,581,0.994063,97854,97854,865.172381,107.611129,46.198889,9144,9.344629,42736,43.673674,46.198889,890.6250,1664.062500,0.053398,0.124381,70.611200,196.923077,30.476190,10.654971,736.367703,421.928603,1.745242,63.573345,36.426655,1990.267463,831.971157,21.084680,NaN,32.667714,148.637581,4.549984,4.549984,4.890367,2705.194542,NaN,Sinus,0
3,nsr2db-nsr004,nsr2db,62.0,F,128,NaN,NaN,NaN,97779,64,0.999346,97842,97842,850.729445,172.722482,30.805785,8513,8.700851,41323,42.234850,30.805786,843.7500,1207.031250,0.036211,0.203029,73.669548,196.923077,39.689922,15.929262,714.416160,262.515948,2.721420,73.128537,26.871463,1933.274660,956.342552,34.008342,NaN,21.783091,243.293256,11.168904,11.168904,4.928370,10869.276197,NaN,Sinus,0
4,nsr2db-nsr005,nsr2db,62.0,F,128,NaN,NaN,NaN,116261,1033,0.991193,117293,117293,733.694180,221.550335,32.689939,6200,5.285953,39612,33.772124,32.689939,679.6875,1496.093750,0.044555,0.301966,89.109575,199.480519,33.391304,25.261726,1259.575385,203.648133,6.185057,86.082227,13.917773,2771.409102,1308.185583,21.834140,NaN,23.115376,312.465648,13.517654,13.517654,5.062823,16895.209705,NaN,Sinus,0
5,nsr2db-nsr006,nsr2db,64.0,M,128,NaN,NaN,NaN,102620,431,0.995818,103050,103050,806.669927,167.963744,31.154222,2646,2.567711,26687,25.897389,31.154222,835.9375,1644.531250,0.038621,0.208219,78.277332,199.480519,30.843373,19.305681,828.984763,190.137039,4.359933,81.343051,18.656949,1761.326632,742.204831,32.203125,NaN,22.029468,236.512878,10.736205,10.736205,4.920979,10157.002522,NaN,Sinus,0
6,nsr2db-nsr007,nsr2db,76.0,M,128,NaN,NaN,NaN,106702,779,0.992752,107480,107480,786.528235,107.585783,41.329094,1690,1.572400,27806,25.871100,41.329094,789.0625,1699.218750,0.052546,0.136786,78.007762,199.480519,30.000000,13.396506,681.793431,315.684604,2.159730,68.351724,31.648276,1545.334328,547.856293,19.779168,NaN,29.224219,149.316264,5.109333,5.109333,4.843970,3051.626035,NaN,Sinus,0
7,nsr2db-nsr008,nsr2db,64.0,F,128,NaN,NaN,NaN,107665,381,0.996474,108045,108045,794.690472,147.346580,34.902264,5904,5.464440,32964,30.509792,34.902264,765.6250,1371.093750,0.043919,0.185414,78.137279,199.480519,35.887850,14.812643,987.308842,258.411607,3.820683,79.256052,20.743948,2244.151944,998.431495,32.445946,NaN,24.679742,206.912879,8.383916,8.383916,4.912248,6938.960738,NaN,Sinus,0
8,nsr2db-nsr009,nsr2db,66.0,M,128,NaN,NaN,NaN,102829,45,0.999563,102873,102873,836.051643,167.950087,41.501479,11207,10.894121,38002,36.941053,41.501479,804.6875,1246.093750,0.049640,0.200885,74.671396,199.480519,38.787879,14.903639,1924.390666,380.755886,5.054132,83.482357,16.517643,3671.471186,1366.324634,30.874250,NaN,29.346120,235.697409,8.031638,8.031638,5.044025,7572.145042,NaN,Sinus,0
9,nsr2db-nsr010,nsr2db,61.0,F,128,NaN,NaN,NaN,96045,246,0

In [23]:
pd.read_excel("df_nsr2db.xlsx")

,id,db_source,age,gender,fs,signal_length,recording_time_hours,recording_time_seconds,normal_beats,abnormal_beats,normal_beats_ratio,rri_length,nni_length,mean_nni,sdnn,sdsd,nni_50,pnni_50,nni_20,pnni_20,rmssd,median_nni,range_nni,cvsd,cvnni,mean_hr,max_hr,min_hr,std_hr,lf,hf,lf_hf_ratio,lfnu,hfnu,total_power,vlf,triangular_index,tinn,sd1,sd2,ratio_sd2_sd1,csi,cvi,Modified_csi,sampen,cardiac_info,risk
0,nsr2db-nsr001,nsr2db,64.0,F,128,NaN,NaN,NaN,106379,456,0.995732,106834,106834,758.344429,170.819356,39.250470,9754,9.130138,37691,35.280297,39.250470,703.1250,1167.968750,0.051758,0.225253,83.057407,199.480519,40.851064,18.258777,778.630879,428.132798,1.818667,64.522234,35.477766,2330.796029,1124.032351,35.096583,NaN,27.754403,239.975411,8.646391,8.646391,5.027619,8299.684610,NaN,Sinus,0
1,nsr2db-nsr002,nsr2db,67.0,M,128,NaN,NaN,NaN,111100,284,0.997450,111383,111382,754.286171,129.156802,19.693105,1410,1.265925,21823,19.593108,19.693105,742.1875,1445.312500,0.026108,0.171231,81.876288,199.480519,34.362416,13.870781,523.537577,49.767639,10.519639,91.319172,8.680828,1310.427230,737.122013,29.411672,NaN,13.925191,182.123716,13.078723,13.078723,4.608288,9527.782830,NaN,Sinus,0
2,nsr2db-nsr003,nsr2db,67.0,F,128,NaN,NaN,NaN,97274,581,0.994063,97854,97854,865.172381,107.611129,46.198889,9144,9.344629,42736,43.673674,46.198889,890.6250,1664.062500,0.053398,0.124381,70.611200,196.923077,30.476190,10.654971,736.367703,421.928603,1.745242,63.573345,36.426655,1990.267463,831.971157,21.084680,NaN,32.667714,148.637581,4.549984,4.549984,4.890367,2705.194542,NaN,Sinus,0
3,nsr2db-nsr004,nsr2db,62.0,F,128,NaN,NaN,NaN,97779,64,0.999346,97842,97842,850.729445,172.722482,30.805785,8513,8.700851,41323,42.234850,30.805786,843.7500,1207.031250,0.036211,0.203029,73.669548,196.923077,39.689922,15.929262,714.416160,262.515948,2.721420,73.128537,26.871463,1933.274660,956.342552,34.008342,NaN,21.783091,243.293256,11.168904,11.168904,4.928370,10869.276197,NaN,Sinus,0
4,nsr2db-nsr005,nsr2db,62.0,F,128,NaN,NaN,NaN,116261,1033,0.991193,117293,117293,733.694180,221.550335,32.689939,6200,5.285953,39612,33.772124,32.689939,679.6875,1496.093750,0.044555,0.301966,89.109575,199.480519,33.391304,25.261726,1259.575385,203.648133,6.185057,86.082227,13.917773,2771.409102,1308.185583,21.834140,NaN,23.115376,312.465648,13.517654,13.517654,5.062823,16895.209705,NaN,Sinus,0
5,nsr2db-nsr006,nsr2db,64.0,M,128,NaN,NaN,NaN,102620,431,0.995818,103050,103050,806.669927,167.963744,31.154222,2646,2.567711,26687,25.897389,31.154222,835.9375,1644.531250,0.038621,0.208219,78.277332,199.480519,30.843373,19.305681,828.984763,190.137039,4.359933,81.343051,18.656949,1761.326632,742.204831,32.203125,NaN,22.029468,236.512878,10.736205,10.736205,4.920979,10157.002522,NaN,Sinus,0
6,nsr2db-nsr007,nsr2db,76.0,M,128,NaN,NaN,NaN,106702,779,0.992752,107480,107480,786.528235,107.585783,41.329094,1690,1.572400,27806,25.871100,41.329094,789.0625,1699.218750,0.052546,0.136786,78.007762,199.480519,30.000000,13.396506,681.793431,315.684604,2.159730,68.351724,31.648276,1545.334328,547.856293,19.779168,NaN,29.224219,149.316264,5.109333,5.109333,4.843970,3051.626035,NaN,Sinus,0
7,nsr2db-nsr008,nsr2db,64.0,F,128,NaN,NaN,NaN,107665,381,0.996474,108045,108045,794.690472,147.346580,34.902264,5904,5.464440,32964,30.509792,34.902264,765.6250,1371.093750,0.043919,0.185414,78.137279,199.480519,35.887850,14.812643,987.308842,258.411607,3.820683,79.256052,20.743948,2244.151944,998.431495,32.445946,NaN,24.679742,206.912879,8.383916,8.383916,4.912248,6938.960738,NaN,Sinus,0
8,nsr2db-nsr009,nsr2db,66.0,M,128,NaN,NaN,NaN,102829,45,0.999563,102873,102873,836.051643,167.950087,41.501479,11207,10.894121,38002,36.941053,41.501479,804.6875,1246.093750,0.049640,0.200885,74.671396,199.480519,38.787879,14.903639,1924.390666,380.755886,5.054132,83.482357,16.517643,3671.471186,1366.324634,30.874250,NaN,29.346120,235.697409,8.031638,8.031638,5.044025,7572.145042,NaN,Sinus,0
9,nsr2db-nsr010,nsr2db,61.0,F,128,NaN,NaN,NaN,96045,246,0

## Extensive Data Preprocessing

1. Look at [1] Tabel 1 for data acquisition information & Tabel 6-7 for preprocessing methods
2. Rule for removing/omitting observations
3. Interpolation method
4. Regarding ectopic beat
5. Checking the annotation of the R-peak (Normal or else). Maybe only use L consecutives of normal-labeled R-peaks or something for short-term
6. Other good references [2] Figure 2, [3] section 2 "Materials and methods"


```
[1] Alan Jovic et al, "Detection of congestive heart failure from short-term heart rate variability segments using hybrid feature selection approach"
[2] Liang Zhou et al, "Automatic Detection of Congestive Heart Failure Based on Multiscale Residual UNet++: From Centralized Learning to Federated Learning"
[3] Annisa Darmawahyuni et al, "Congestive heart failure waveform classification based on short time-step analysis with recurrent network"
```